<a href="https://colab.research.google.com/github/SConstant/C7804-Big-Data-and-Decision-Making/blob/main/Data_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import pandas as pd
import requests
import urllib.parse

# Importing Dataset

In [ ]:
# Import the dataset from Github
# Set URL

url = "https://raw.githubusercontent.com/SConstant/C7804-Big-Data-and-Decision-Making/main/IndianCommodities.csv"



In [ ]:
# Create a dataframe to run the geocoding

df = pd.read_csv(url, encoding="ISO-8859-1")

In [ ]:
# Create a dataframe to eventually attache data

ndf = pd.read_csv(url, encoding="ISO-8859-1")


In [ ]:
# Explore

ndf.head

<bound method NDFrame.head of         Unnamed: 0        Date     Market               Category  \
0                1  1994-01-15      Delhi     cereals and tubers   
1                2  1994-01-15      Delhi     cereals and tubers   
2                3  1994-01-15      Delhi     miscellaneous food   
3                4  1994-01-15      Delhi           oil and fats   
4                5  1994-01-15  Ahmedabad     cereals and tubers   
...            ...         ...        ...                    ...   
139529      139530  2021-09-15  Kharagpur        pulses and nuts   
139530      139531  2021-09-15  Kharagpur        pulses and nuts   
139531      139532  2021-09-15  Kharagpur        pulses and nuts   
139532      139533  2021-09-15  Kharagpur  vegetables and fruits   
139533      139534  2021-09-15  Kharagpur  vegetables and fruits   

              Commodity Unit PriceFlag PriceType Currency  Price  USD_Price  
0                  Rice   KG    actual    Retail      INR    8.0     0.2545

In [ ]:
df = df.copy()

In [ ]:
# Select Market rows and put in new dataframe object

df = df[['Market']]

In [ ]:
# Drop duplicates to make computationally less expensive and a bit faster

df = df.drop_duplicates()

In [ ]:
# Explore

df.head

<bound method NDFrame.head of             Market
0            Delhi
4        Ahmedabad
8           Shimla
11       Bengaluru
14          Bhopal
...            ...
136823   Dantewada
136970     Selamba
137053      Bodeli
137326     Dhanbad
137389  Jamshedpur

[165 rows x 1 columns]>

# Geocoding dataframe

In [ ]:
# Open Street map
# Test out on one row, and index 0
# Put in the API as URL, and parse the Market data alongside calling the json via the url. 
# Test by printing the Latitude and Longitude

url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(df['Market'][0]) +'?format=json'
response = requests.get(url).json()
print('Latitude: '+response[0]['lat']+', Longitude: '+response[0]['lon'])

Latitude: 28.6517178, Longitude: 77.2219388


In [ ]:
# Apply to the entire dataframe df
# As above with additional if/else code where if the Latitude and Longitude are available populate or return a -1 value
# Add this data as a new column to df

def geocode2(Market):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(Market) +'?format=json'
    response = requests.get(url).json()
    if(len(response)!=0):
        return(response[0]['lat'], response[0]['lon'])
    else:
        return('-1')

df['geocoded'] = df['Market'].apply(geocode2)

In [ ]:
df.head

# Merging Geocoded data to orginal dataset

In [ ]:
# Merging the Geocoded dataframe, with the original dataframe

ndf = pd.merge(ndf, df, how='inner', on='Market')

In [ ]:
# Pretty Pandas display to explore what the data looks like, and to be able to look closely follow the previosu operation 

with pd.option_context('display.max_rows', 165,
                       'display.max_columns', None,
                       'display.width', 1000,
                       'display.precision', 3,
                       'display.colheader_justify', 'center'):
  
  display(df)

# Tidying the dataframe

In [ ]:
# How many rows are -1

ndf_rows = ndf.apply(lambda x : True
            if x['geocoded'] == "-1" else False, axis = 1)

In [ ]:
num_rows = len(ndf_rows[ndf_rows == True].index)

In [ ]:
print('Number of Rows in dataframe in which geocoded is -1 : ',
      num_rows )

In [ ]:
ndf.head

In [ ]:
# Dropping the -1 values, these are values which open street map couldn't find the lat and long data for

ndf[ndf.geocoded != '-1']

In [ ]:
# Splitting the geocoded row into Lat and long

ndf['Lat'] = ndf.geocoded.str.split(',', expand = True)[0]
ndf['Long'] = ndf.geocoded.str.split(',', expand = True)[1]

In [ ]:
ndf.shape


# Changing date to integer 

In [ ]:
# Changing Date to integer 

ndf["Date"] = p.to_datetime(ndf["Date"])

# Saving to CSV

In [ ]:
# writing ndf to csv

ndf.to_csv('indian_commodities_geo.csv', index=False)